In [1]:
!pip install torch_geometric > /dev/null
# !pip install qiskit > /dev/null
# !pip install qiskit_aer > /dev/null
# !pip install pennylane
# !pip install qiskit-aer-gpu

In [2]:
# !pip install --upgrade "pennylane-lightning[gpu]"  # pulls pennylane + plugin
# !pip install --upgrade custatevec-cu12           # CUDA 11 build of cuQuantum

In [3]:
import concurrent.futures

import matplotlib.pyplot as plt

import numpy as np

import os

import pandas as pd

# import pennylane as qml
# from pennylane import DeviceError



# from qiskit import QuantumCircuit
# from qiskit.circuit import ParameterVector
# from qiskit.quantum_info import SparsePauliOp, Statevector
# from qiskit_aer import AerSimulator
# from qiskit_aer.primitives import EstimatorV2

from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
from torch.func import vmap  
import torch.nn as nn
from torch.nn import Linear
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import torch_geometric
from torch_geometric.datasets import JODIEDataset
from torch_geometric.loader import TemporalDataLoader
from torch_geometric.nn.conv import TransformerConv
from torch_geometric.nn.models import tgn
from torch_geometric.nn.models.tgn import TGNMemory, IdentityMessage, LastNeighborLoader, LastAggregator

from tqdm.notebook import tqdm

In [4]:
# data = pd.read_csv("/kaggle/input/dataset-invade/dataset_invade.csv")
# features = data.drop(columns=['attack'])
# target = data['attack'].apply(lambda x: 1 if x == 'Yes' else 0)

# # Convert categorical columns to one-hot encoded columns
# non_numeric_columns = features.select_dtypes(include=['object']).columns
# features = pd.get_dummies(features, columns=non_numeric_columns)

# # Convert boolean columns to integers
# for col in features.select_dtypes(include=['bool']).columns:
#     features[col] = features[col].astype(int)

# scaler = StandardScaler()
# features = scaler.fit_transform(features)

# # Convert to PyTorch tensors
# X_tensor = torch.tensor(features, dtype=torch.float32)
# y_tensor = torch.tensor(target, dtype=torch.float32)

# # Display the shapes of the tensors
# print(X_tensor.shape, y_tensor.shape)

# # Split the data into training, validation, and test sets
# X_train, X_temp, y_train, y_temp = train_test_split(X_tensor, y_tensor, test_size=0.3, random_state=42)
# X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# # Create DataLoader for each dataset
# batch_size = 64

# train_dataset = TensorDataset(X_train, y_train)
# val_dataset = TensorDataset(X_val, y_val)
# test_dataset = TensorDataset(X_test, y_test)

# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# This one i think


In [5]:
# class SineLayer(torch.nn.Module):
#     def __init__(self, input_dim, output_dim):
#         super(QuantumLayer, self).__init__()
#         self.linear = torch.nn.Linear(input_dim, output_dim)

#     def forward(self, x):
#         x = self.linear(x)
#         x = torch.sin(x)  # Apply a quantum-inspired non-linearity
#         return x

# class SineNN(torch.nn.Module):
#     def __init__(self, input_dim):
#         super(QuantumNN, self).__init__()
#         self.sine_layer1 = SineLayer(input_dim, 128)
#         self.sine_layer2 = SineLayer(128, 64)
#         self.fc = torch.nn.Linear(64, 1)
#         self.dropout = torch.nn.Dropout(0.5)

#     def forward(self, x):
#         x = torch.relu(self.sine_layer1(x))
#         x = self.dropout(x)
#         x = torch.relu(self.sine_layer2(x))
#         x = self.dropout(x)
#         x = torch.sigmoid(self.fc(x))
#         return x

In [6]:
# class SimplerFNN(torch.nn.Module):
#     def __init__(self, input_dim):
#         super().__init__()
#         self.model = torch.nn.Sequential(
#             torch.nn.Linear(input_dim, 64),
#             torch.nn.ReLU(),
#             torch.nn.Linear(64, 32),
#             torch.nn.ReLU(),
#             torch.nn.Linear(32, 1),
#             torch.nn.Sigmoid()
#         )

#     def forward(self, x):
#         return self.model(x)


In [7]:
# # Training function
# def train_model(model, optimizer, loader):
#     model.train()
#     running_loss = 0.0
#     for inputs, labels in loader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs.squeeze(), labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item() * inputs.size(0)
#     return running_loss / len(loader.dataset)

# # Evaluation function
# def evaluate_model(model, optimizer, loader):
#     model.eval()
#     running_loss = 0.0
#     all_outputs = []
#     all_labels = []
#     with torch.no_grad():
#         for inputs, labels in loader:
#             outputs = model(inputs)
#             loss = criterion(outputs.squeeze(), labels)
#             running_loss += loss.item() * inputs.size(0)
#             all_outputs.append(outputs.squeeze().cpu())
#             all_labels.append(labels.cpu())
#     all_outputs = torch.cat(all_outputs)
#     all_labels = torch.cat(all_labels)
#     return running_loss / len(loader.dataset), all_outputs, all_labels

In [8]:
# def run_it(num_epochs, model, optimizer, train_loader, val_loader, test_loader):
#     for epoch in range(1, num_epochs + 1):
#         if epoch > 30:
#             if sum(val_losses[-20:-10]) < sum(val_losses[-10:]):
#                 print("Early Stopping")
#                 break
#         train_loss = train_model(model, optimizer, train_loader)
#         val_loss, val_outputs, val_labels = evaluate_model(model, optimizer, val_loader)
#         test_loss, test_outputs, test_labels = evaluate_model(model, optimizer, test_loader)
    
#         val_ap = average_precision_score(val_labels, val_outputs)
#         val_auc = roc_auc_score(val_labels, val_outputs)
#         test_ap = average_precision_score(test_labels, test_outputs)
#         test_auc = roc_auc_score(test_labels, test_outputs)
    
#         train_losses.append(train_loss)
#         val_losses.append(val_loss)
#         val_aps.append(val_ap)
#         val_aucs.append(val_auc)
#         test_aps.append(test_ap)
#         test_aucs.append(test_auc)
    
#         print(f'Epoch: {epoch:02d}, Loss: {train_loss:.4f} Val AP: {val_ap:.4f}, Val AUC: {val_auc:.4f} Test AP: {test_ap:.4f}, Test AUC: {test_auc:.4f}')



In [9]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# input_dim = X_tensor.shape[1]
# criterion = torch.nn.BCELoss()
# model = SimplerFNN(input_dim)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



# # Training loop
# num_epochs = 200
# train_losses = []
# val_losses = []
# val_aps = []
# val_aucs = []
# test_aps = []
# test_aucs = []

# run_it(num_epochs, model, optimizer, train_loader, val_loader, test_loader)



## Output from cell above

```Epoch: 01, Loss: 0.0933 Val AP: 0.9980, Val AUC: 0.9980 Test AP: 0.9978, Test AUC: 0.9982
Epoch: 02, Loss: 0.0498 Val AP: 0.9984, Val AUC: 0.9984 Test AP: 0.9984, Test AUC: 0.9986
Epoch: 03, Loss: 0.0444 Val AP: 0.9986, Val AUC: 0.9987 Test AP: 0.9985, Test AUC: 0.9987
Epoch: 04, Loss: 0.0419 Val AP: 0.9987, Val AUC: 0.9987 Test AP: 0.9987, Test AUC: 0.9988
Epoch: 05, Loss: 0.0397 Val AP: 0.9988, Val AUC: 0.9989 Test AP: 0.9988, Test AUC: 0.9989
Epoch: 06, Loss: 0.0386 Val AP: 0.9987, Val AUC: 0.9988 Test AP: 0.9987, Test AUC: 0.9989
Epoch: 07, Loss: 0.0372 Val AP: 0.9987, Val AUC: 0.9989 Test AP: 0.9988, Test AUC: 0.9990
Epoch: 08, Loss: 0.0363 Val AP: 0.9988, Val AUC: 0.9990 Test AP: 0.9988, Test AUC: 0.9990
Epoch: 09, Loss: 0.0352 Val AP: 0.9988, Val AUC: 0.9990 Test AP: 0.9989, Test AUC: 0.9991
Epoch: 10, Loss: 0.0340 Val AP: 0.9988, Val AUC: 0.9989 Test AP: 0.9989, Test AUC: 0.9990
Epoch: 11, Loss: 0.0338 Val AP: 0.9988, Val AUC: 0.9989 Test AP: 0.9988, Test AUC: 0.9990
Epoch: 12, Loss: 0.0325 Val AP: 0.9990, Val AUC: 0.9991 Test AP: 0.9989, Test AUC: 0.9991
Epoch: 13, Loss: 0.0339 Val AP: 0.9990, Val AUC: 0.9991 Test AP: 0.9989, Test AUC: 0.9991
Epoch: 14, Loss: 0.0347 Val AP: 0.9991, Val AUC: 0.9991 Test AP: 0.9989, Test AUC: 0.9992
Epoch: 15, Loss: 0.0340 Val AP: 0.9990, Val AUC: 0.9991 Test AP: 0.9989, Test AUC: 0.9991
Epoch: 16, Loss: 0.0335 Val AP: 0.9991, Val AUC: 0.9992 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 17, Loss: 0.0332 Val AP: 0.9991, Val AUC: 0.9991 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 18, Loss: 0.0327 Val AP: 0.9990, Val AUC: 0.9991 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 19, Loss: 0.0332 Val AP: 0.9990, Val AUC: 0.9991 Test AP: 0.9989, Test AUC: 0.9992
Epoch: 20, Loss: 0.0328 Val AP: 0.9990, Val AUC: 0.9991 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 21, Loss: 0.0318 Val AP: 0.9990, Val AUC: 0.9991 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 22, Loss: 0.0313 Val AP: 0.9991, Val AUC: 0.9992 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 23, Loss: 0.0308 Val AP: 0.9991, Val AUC: 0.9991 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 24, Loss: 0.0303 Val AP: 0.9991, Val AUC: 0.9991 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 25, Loss: 0.0299 Val AP: 0.9991, Val AUC: 0.9991 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 26, Loss: 0.0298 Val AP: 0.9991, Val AUC: 0.9992 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 27, Loss: 0.0293 Val AP: 0.9991, Val AUC: 0.9991 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 28, Loss: 0.0294 Val AP: 0.9991, Val AUC: 0.9992 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 29, Loss: 0.0286 Val AP: 0.9992, Val AUC: 0.9992 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 30, Loss: 0.0289 Val AP: 0.9991, Val AUC: 0.9991 Test AP: 0.9989, Test AUC: 0.9991
Epoch: 31, Loss: 0.0284 Val AP: 0.9992, Val AUC: 0.9992 Test AP: 0.9991, Test AUC: 0.9993
Epoch: 32, Loss: 0.0285 Val AP: 0.9991, Val AUC: 0.9991 Test AP: 0.9991, Test AUC: 0.9993
Epoch: 33, Loss: 0.0281 Val AP: 0.9989, Val AUC: 0.9991 Test AP: 0.9990, Test AUC: 0.9992
Epoch: 34, Loss: 0.0283 Val AP: 0.9991, Val AUC: 0.9992 Test AP: 0.9991, Test AUC: 0.9993
Epoch: 35, Loss: 0.0276 Val AP: 0.9990, Val AUC: 0.9991 Test AP: 0.9991, Test AUC: 0.9993
Epoch: 36, Loss: 0.0274 Val AP: 0.9990, Val AUC: 0.9991 Test AP: 0.9991, Test AUC: 0.9992
Epoch: 37, Loss: 0.0270 Val AP: 0.9990, Val AUC: 0.9991 Test AP: 0.9991, Test AUC: 0.9993
Early Stopping```

# Multiclass

In [10]:
data_path = "/kaggle/input/network-intrusion-dataset/"
data_files = os.listdir("/kaggle/input/network-intrusion-dataset/")

# Load and concatenate all CSV files into a single DataFrame
data_list = []
for file in data_files:
    if file.endswith('.csv'):
        file_path = os.path.join(data_path, file)
        data_list.append(pd.read_csv(file_path))

data = pd.concat(data_list, ignore_index=True)
print("Data loaded successfully!")


Data loaded successfully!


In [11]:
# Strip any leading/trailing whitespace characters from the column names
data.columns = data.columns.str.strip()

# Drop columns that won't be used for training (like non-numeric features)
columns_to_drop = ['Flow ID', 'Source IP', 'Source Port', 'Destination IP', 'Timestamp']
columns_to_drop = [col.strip() for col in columns_to_drop if col.strip() in data.columns]
data = data.drop(columns=columns_to_drop)

# Convert categorical columns to one-hot encoded columns
non_numeric_columns = data.select_dtypes(include=['object']).columns
data = pd.get_dummies(data, columns=non_numeric_columns)

# Handle infinity or extremely large values
data = data.replace([np.inf, -np.inf], np.nan).dropna()

# Identify the target columns
label_columns = [col for col in data.columns if 'Label_' in col]

# Combine one-hot encoded label columns into a single target column
data['Label'] = data[label_columns].idxmax(axis=1).apply(lambda x: x.split('_')[1])

# Map the labels to unique integers for multi-class classification
label_mapping = {label: idx for idx, label in enumerate(data['Label'].unique())}
data['Label'] = data['Label'].map(label_mapping)

# Drop the individual label columns
data = data.drop(columns=label_columns)

# Separate features and target
features = data.drop(columns=['Label'])
target = data['Label']

In [26]:
data['Label'].value_counts()

Label
0     2271320
12     230124
1      158804
7      128025
13      10293
5        7935
6        5897
10       5796
11       5499
8        1956
2        1507
3         652
9          36
4          21
14         11
Name: count, dtype: int64

tensor([ 0,  0,  0,  ..., 12,  0, 12])

In [12]:
# Feature scaling
scaler = StandardScaler()
features = scaler.fit_transform(features)

# Convert to PyTorch tensors
X_tensor = torch.tensor(features, dtype=torch.float32)
y_tensor = torch.tensor(target.values, dtype=torch.long)

# Split the data into training, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X_tensor, y_tensor, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Create DataLoader for each dataset
batch_size = 1024

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True, num_workers=4, pin_memory=True)
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=True, num_workers=4, pin_memory=True)

# val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=True, num_workers=4, pin_memory=True)

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [36]:
targ_val = y_train
vals = torch.unique(targ_val, return_counts=True)[0]
cnts = torch.unique(targ_val, return_counts=True)[1]
for val, cnt in zip(vals, cnts):
    print(f'{val}: {cnt}')

0: 1590392
1: 111054
2: 1099
3: 427
4: 16
5: 5574
6: 4109
7: 89542
8: 1400
9: 28
10: 4039
11: 3841
12: 160857
13: 7130
14: 5


In [37]:
targ_val = y_val
vals = torch.unique(targ_val, return_counts=True)[0]
cnts = torch.unique(targ_val, return_counts=True)[1]
for val, cnt in zip(vals, cnts):
    print(f'{val}: {cnt}')

0: 340317
1: 24056
2: 190
3: 97
4: 2
5: 1185
6: 907
7: 19239
8: 278
9: 4
10: 881
11: 833
12: 34660
13: 1528
14: 4


In [38]:
targ_val = y_test
vals = torch.unique(targ_val, return_counts=True)[0]
cnts = torch.unique(targ_val, return_counts=True)[1]
for val, cnt in zip(vals, cnts):
    print(f'{val}: {cnt}')

0: 340611
1: 23694
2: 218
3: 128
4: 3
5: 1176
6: 881
7: 19244
8: 278
9: 4
10: 876
11: 825
12: 34607
13: 1635
14: 2


In [13]:
# Define the model, I changed the shape of the network, why scale from 79 features to a 128 wide model?
class SimpleNN(torch.nn.Module):
    def __init__(self, input_dim, num_classes):
        super().__init__()
        self.model = torch.nn.Sequential(
            torch.nn.Linear(input_dim, 64),
            torch.nn.ReLU(),
            torch.nn.Linear(64, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, num_classes),
            # torch.nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)
# Training function
scaler = torch.amp.GradScaler('cuda')

def train_model(loader):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(loader):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        with torch.amp.autocast('cuda'):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        running_loss += loss.item() * inputs.size(0)
        # print("OUTPUT SHAPE:", outputs.shape)
        # print("LABEL SHAPE:", labels.shape)
    return running_loss / len(loader.dataset)


# Evaluation function
def evaluate_model(loader):
    model.eval()
    running_loss = 0.0
    all_outputs = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)  # raw logits
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)

            probs = F.softmax(outputs, dim=1)  # convert to probabilities
            all_outputs.append(probs.cpu())
            all_labels.append(labels.cpu())

    all_outputs = torch.cat(all_outputs)
    all_labels = torch.cat(all_labels)
    return running_loss / len(loader.dataset), all_outputs, all_labels


/usr/local/lib/python3.11/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [14]:

input_dim = X_tensor.shape[1]
NUM_CLASSES = len(label_mapping)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = torch.nn.CrossEntropyLoss()
model = SimpleNN(input_dim=input_dim, num_classes=NUM_CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# Quantum Tests


In [15]:
# class HybridNN(nn.Module):
#     def __init__(self, input_dim, num_classes):
#         super().__init__()

#         self.scale = nn.Parameter(torch.ones(8) * np.pi)

#         self.classical = nn.Sequential(
#             nn.Linear(input_dim, 64),
#             nn.ReLU(),
#             nn.Linear(64, 32),
#             nn.ReLU(),
#             nn.Linear(32, 8),
#             nn.ReLU()
#         )

#         self.output_layer = nn.Sequential(
#             nn.Linear(8, 64),
#             nn.ReLU(),
#             nn.Linear(64, 32),
#             nn.ReLU(),
#             nn.Linear(32, num_classes)
#         )

#         self.simulator = AerSimulator(method='statevector', device = 'CPU')
#         self.estimator = EstimatorV2.from_backend(self.simulator)

#         # Create parameterized circuit (once)
#         self.params = ParameterVector("θ", 8)
#         self.qc_template = QuantumCircuit(8)
#         for i in range(8):
#             self.qc_template.ry(self.params[i], i)
#         for i in range(8):
#             self.qc_template.cz(i, (i + 1) % 8)

#         # Observables: Z_i for each qubit
#         self.observables = [
#             SparsePauliOp.from_list([(f"{'I'*i}Z{'I'*(7-i)}", 1.0)])
#             for i in range(8)
#         ]

#     def quantum_layer(self, x_batch):
#         """
#         x_batch: Tensor of shape (batch_size, 8)
#         Returns: Tensor of shape (batch_size, 8) with Z expectation values
#         """
#         # Convert scaled inputs to list of NumPy arrays
#         batch_scaled = (self.scale * x_batch).detach().cpu().numpy()
    
#         # Map each input to param dict (ParameterVector already set in init)
#         param_list = [dict(zip(self.params, row)) for row in batch_scaled]
    
#         # Create one circuit (will be copied internally)
#         jobs = [(self.qc_template.assign_parameters(p, inplace=False), self.observables) for p in param_list]
    
#         # Run batch job
#         result = self.estimator.run(jobs).result()
    
#         # Stack results back into tensor
#         evs = [torch.tensor(r.data.evs, dtype=torch.float32) for r in result]
#         return torch.stack(evs).to(x_batch.device)

#     def forward(self, x):
#         x = self.classical(x)             # shape: (batch_size, 8)
#         x = self.quantum_layer(x)         # shape: (batch_size, 8)
#         x = self.output_layer(x)          # shape: (batch_size, num_classes)
#         return x

In [16]:
# # Constants
# scale = np.array([np.pi for _ in range(8)])
# simulator = AerSimulator(method='statevector')
# estimator = EstimatorV2.from_backend(simulator)

# # Individual observables: Z_i for i=0..7
# observables = [SparsePauliOp.from_list([(f"{'I'*i}Z{'I'*(7-i)}", 1.0)]) for i in range(8)]

# # Input: shape (8, 1024)
# x = np.random.rand(8, 1024)
# scaled = scale[:, None] * x

# results = []

# for col in range(scaled.shape[1]):
#     theta_sample = scaled[:, col]

#     qc = QuantumCircuit(8)
#     for q in range(8):
#         qc.ry(theta_sample[q], q)
#     for q in range(8):
#         qc.cz(q, (q + 1) % 8)

#     job = estimator.run([(qc, observables)])
#     result = job.result()
#     results.append(result[0].data.evs)

# # Result: shape (1024, 8)
# result_array = np.array(results)
# print(result_array.shape)
# print("First result:", result_array[0])

In [17]:
# class HybridNN(nn.Module):
#     def __init__(self, input_dim, num_classes, n_qubits=8):
#         super().__init__()
#         self.n_qubits = n_qubits
#         self.scale = nn.Parameter(torch.ones(n_qubits) * np.pi)

#         self.classical = nn.Sequential(
#             nn.Linear(input_dim, 64),
#             nn.ReLU(),
#             nn.Linear(64, 32),
#             nn.ReLU(),
#             nn.Linear(32, n_qubits),
#             nn.ReLU()
#         )

#         self.output_layer = nn.Sequential(
#             nn.Linear(n_qubits, 16),
#             nn.ReLU(),
#             nn.Linear(16, num_classes)
#         )

#         # Precompute Z_i matrices for matrix inner product method
#         self.register_buffer("Z_ops", self._precompute_z_ops(n_qubits))

#     def _precompute_z_ops(self, n_qubits):
#         Z = torch.tensor([[1, 0], [0, -1]], dtype=torch.complex64)
#         I = torch.eye(2, dtype=torch.complex64)
#         Z_ops = []

#         for i in range(n_qubits):
#             op = None
#             for j in range(n_qubits):
#                 mat = Z if j == i else I
#                 op = mat if op is None else torch.kron(op, mat)
#             Z_ops.append(op)
#         return torch.stack(Z_ops)  # shape: (n_qubits, 2**n, 2**n)

#     def quantum_layer(self, x_batch):
#         """
#         x_batch: Tensor of shape (batch_size, n_qubits)
#         Returns: Tensor of shape (batch_size, n_qubits)
#         """
#         device = x_batch.device
#         batch_size, n_qubits = x_batch.shape
#         results = []
    
#         for b in range(batch_size):
#             x = x_batch[b].detach().cpu().numpy() * self.scale.detach().cpu().numpy()
    
#             # Build and simulate circuit
#             qc = QuantumCircuit(n_qubits)
#             for i in range(n_qubits):
#                 qc.ry(x[i], i)
#             for i in range(n_qubits):
#                 qc.cz(i, (i + 1) % n_qubits)
    
#             psi_np = Statevector.from_instruction(qc).data
#             psi = torch.tensor(psi_np, dtype=torch.complex64, device=device)  # shape: (2**n,)
    
#             # Compute expectation values ⟨ψ|Z_i|ψ⟩
#             evs = torch.einsum('i,bij,j->b', psi.conj(), self.Z_ops.to(device), psi)
#             results.append(evs.real)
    
#         return torch.stack(results)  # (batch_size, n_qubits)



#     def forward(self, x):
#         x = self.classical(x)             # shape: (batch_size, 8)
#         x = self.quantum_layer(x)         # shape: (batch_size, 8)
#         x = self.output_layer(x)          # shape: (batch_size, num_classes)
#         return x

In [18]:
# class TorchQuantumLayer(nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.num_qubits = 8
#         self.scale = nn.Parameter(torch.ones(self.num_qubits) * torch.pi)

#         # Define Pauli-Z operators (I⊗I⊗...⊗Z⊗...⊗I) in matrix form
#         Z = torch.tensor([[1., 0.], [0., -1.]], dtype=torch.float32)
#         I = torch.eye(2, dtype=torch.float32)

#         self.paulis = []
#         for i in range(self.num_qubits):
#             ops = [I] * self.num_qubits
#             ops[i] = Z
#             pauli = ops[0]
#             for op in ops[1:]:
#                 pauli = torch.kron(pauli, op)
#             self.paulis.append(pauli)

#         self.paulis = torch.stack(self.paulis).to("cuda" if torch.cuda.is_available() else "cpu")

#     def forward(self, x):
#         """
#         x: (batch_size, 8)
#         Returns: (batch_size, 8) expectation values
#         """
#         device = x.device
#         batch_size = x.shape[0]
#         angles = x * self.scale  # shape: (batch_size, 8)

#         # Build quantum states for each sample: apply Ry(θ) to |0⟩^⊗n
#         zero_state = torch.tensor([1.0, 0.0], dtype=torch.float32, device=device)
#         full_zero = zero_state
#         for _ in range(self.num_qubits - 1):
#             full_zero = torch.kron(full_zero, zero_state)  # shape: (2^n,)

#         state_batch = []
#         for sample in angles:
#             state = full_zero.clone()
#             for i in range(self.num_qubits):
#                 theta = sample[i]
#                 ry = torch.tensor([
#                     [torch.cos(theta / 2), -torch.sin(theta / 2)],
#                     [torch.sin(theta / 2), torch.cos(theta / 2)]
#                 ], dtype=torch.float32, device=device)

#                 # Apply Ry gate
#                 eye_prefix = torch.eye(2 ** i, device=device)
#                 eye_suffix = torch.eye(2 ** (self.num_qubits - i - 1), device=device)
#                 U = torch.kron(torch.kron(eye_prefix, ry), eye_suffix)
#                 state = torch.matmul(U, state)
#             state_batch.append(state)

#         state_batch = torch.stack(state_batch)  # (batch_size, 2^n)

#         # Compute ⟨ψ|P|ψ⟩ for each Pauli observable
#         expectations = []
#         for pauli in self.paulis:
#             expvals = torch.einsum('bi,ij,bj->b', state_batch.conj(), pauli, state_batch)
#             expectations.append(expvals.real)

#         return torch.stack(expectations, dim=1)  # (batch_size, 8)

In [19]:


# class HybridNN(nn.Module):
#     def __init__(self, input_dim, num_classes, n_qubits=8):
#         super().__init__()
#         self.n_qubits = n_qubits
#         self.scale = nn.Parameter(torch.ones(n_qubits) * torch.pi)

#         # Classical front‑end
#         self.classical = nn.Sequential(
#             nn.Linear(input_dim, 64),
#             nn.ReLU(),
#             nn.Linear(64, n_qubits)          # → (B, 8)
#         )

#         # Lightning‑GPU device
#         self.dev = qml.device("lightning.qubit",
#                               wires=n_qubits,
#                               shots=None,        # analytic expectation
#                               batch_obs=True)

#         # Quantum layer wrapped as Torch‑callable
#         self.qnode = qml.QNode(self._circuit,
#                                self.dev,
#                                interface="torch",
#                                diff_method="adjoint")

#         # Classical tail
#         self.output_layer = nn.Sequential(
#             nn.Linear(n_qubits, 16),
#             nn.ReLU(),
#             nn.Linear(16, num_classes)
#         )

#     # ──────────────────────────────────────────────────────────
#     def _circuit(self, data, scale):
#         """Single‑sample, 8‑qubit circuit.
#         data : shape (8,)
#         scale: shape (8,) – trainable
#         """
#         # Data re‑scaling & embedding
#         for i in range(self.n_qubits):
#             qml.RY(scale[i] * data[i], wires=i)

#         # Ring entanglement
#         for i in range(self.n_qubits):
#             qml.CNOT(wires=[i, (i + 1) % self.n_qubits])

#         # Expectation values <Z_i>
#         return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
#     # ──────────────────────────────────────────────────────────

#     def forward(self, x):
#         """
#         x: (B, input_dim) – full batch.
#         PennyLane 0.36+ lets us pass the whole batch at once by
#         giving tensor `x_emb` of shape (B, 8) as the first arg.
#         """
#         x_emb = self.classical(x)                # (B, 8)

#         # Call qnode over the batch; PennyLane will vectorise internally
#         z_expect = self.qnode(x_emb, self.scale) # (B, 8)

#         logits = self.output_layer(z_expect)
#         return logits


In [20]:
# input_dim = X_tensor.shape[1]
# NUM_CLASSES = len(label_mapping)


# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# criterion = torch.nn.CrossEntropyLoss()
# model = HybridNN(input_dim=input_dim, num_classes=NUM_CLASSES).to(device)
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [21]:
def train_model(loader):
    model.train()
    running_loss = 0.0

    for x, y in tqdm(loader):                     # x: (B, input_dim)
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(x)                   # (B, NUM_CLASSES)
        loss   = criterion(logits, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * x.size(0)

    return running_loss / len(loader.dataset)


@torch.no_grad()
def evaluate_model(loader):
    model.eval()
    running_loss, all_logits, all_labels = 0.0, [], []

    for x, y in loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        loss   = criterion(logits, y)

        running_loss += loss.item() * x.size(0)
        all_logits.append(F.softmax(logits, dim=1).cpu())   # probabilities
        all_labels.append(y.cpu())

    all_logits = torch.cat(all_logits)
    all_labels = torch.cat(all_labels)

    return (running_loss / len(loader.dataset),
            all_logits,                 # (N, NUM_CLASSES)
            all_labels)                 # (N,)

# Quantum-only run

In [22]:


# # ─── training loop ─────────────────────────────────────────────
# num_epochs = 30
# best_val_auc, epochs_since_improvement = 0.0, 0
# best_model_path = "best_model.pth"

# train_losses, val_losses, val_aucs, test_aucs = [], [], [], []

# for epoch in range(1, num_epochs + 1):
#     print(f"\nEpoch {epoch}/{num_epochs}")
#     train_loss = train_model(train_loader)
#     val_loss,  val_probs,  val_labels  = evaluate_model(val_loader)
#     test_loss, test_probs, test_labels = evaluate_model(test_loader)

#     # AUC scores
#     val_auc  = roc_auc_score(val_labels.numpy(),
#                              val_probs.numpy(),
#                              multi_class="ovr",
#                              average="macro")
#     test_auc = roc_auc_score(test_labels.numpy(),
#                              test_probs.numpy(),
#                              multi_class="ovr",
#                              average="macro")

#     # bookkeeping
#     train_losses.append(train_loss)
#     val_losses.append(val_loss)
#     val_aucs.append(val_auc)
#     test_aucs.append(test_auc)

#     # always save epoch checkpoint
#     torch.save(model.state_dict(), f"model_epoch_{epoch:02d}.pth")

#     # best‑model logic
#     if val_auc > best_val_auc:
#         best_val_auc = val_auc
#         torch.save(model.state_dict(), best_model_path)
#         epochs_since_improvement = 0
#         print(f"✅  New best model (val AUC {val_auc:.4f})")
#     else:
#         epochs_since_improvement += 1
#         print(f"⚠️   No improvement for {epochs_since_improvement} epoch(s)")

#         if epochs_since_improvement >= 5:
#             # revert & shrink LR
#             model.load_state_dict(torch.load(best_model_path))
#             for g in optimizer.param_groups:
#                 g["lr"] *= 0.5
#             print(f"🔁  Reverted to best model, LR halved to {g['lr']:.6f}")
#             epochs_since_improvement = 0

#     # quick summary
#     print(f"Train {train_loss:.4f} | Val {val_loss:.4f} (AUC {val_auc:.4f}) "
#           f"| Test {test_auc:.4f}")

In [23]:
class CosineQuantumLayer(nn.Module):
    """Fast surrogate for the Ry+CNOT ring producing ⟨Z_i⟩."""
    def __init__(self, n_qubits):
        super().__init__()
        self.scale = nn.Parameter(torch.ones(n_qubits) * torch.pi)

    def forward(self, data):            # data : (B, n_qubits)
        return torch.cos(self.scale * data)   # (B, n_qubits)
class HybridSideBySideFastNN(nn.Module):
    def __init__(self, input_dim, num_classes, n_qubits=8):
        super().__init__()
        self.classical = nn.Sequential(
            nn.Linear(input_dim, 64), nn.ReLU(),
            nn.Linear(64, 32),        nn.ReLU(),
            nn.Linear(32, 16),        nn.ReLU()
        )
        self.q_compress = nn.Linear(16, n_qubits)     # 16 → 8
        self.q_layer    = CosineQuantumLayer(n_qubits)

        self.output = nn.Sequential(
            nn.Linear(n_qubits + 16, 16), nn.ReLU(),
            nn.Linear(16, num_classes)
        )

    def forward(self, x):                          # x : (B, input_dim)
        x_emb  = self.classical(x)                 # (B, 16)
        q_in   = self.q_compress(x_emb)            # (B, 8)
        z_exp  = self.q_layer(q_in)                # (B, 8)

        features = torch.cat([x_emb, z_exp], dim=1)  # (B, 24)
        return self.output(features)


In [24]:
input_dim = X_tensor.shape[1]
NUM_CLASSES = len(label_mapping)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

criterion = torch.nn.CrossEntropyLoss()
model = HybridSideBySideFastNN(input_dim=input_dim, num_classes=NUM_CLASSES).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [25]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report
)

num_epochs = 30
best_val_auc, epochs_since_improvement = 0.0, 0
best_model_path = "best_model.pth"

train_losses, val_losses, val_aucs, test_aucs = [], [], [], []

for epoch in range(1, num_epochs + 1):
    print(f"\nEpoch {epoch}/{num_epochs}")
    train_loss = train_model(train_loader)
    val_loss,  val_probs,  val_labels  = evaluate_model(val_loader)
    test_loss, test_probs, test_labels = evaluate_model(test_loader)


    # AUC scores
    val_auc  = roc_auc_score(val_labels.numpy(),
                             val_probs.numpy(),
                             multi_class="ovr",
                             average="macro")
    test_auc = roc_auc_score(test_labels.numpy(),
                             test_probs.numpy(),
                             multi_class="ovr",
                             average="macro")

    # bookkeeping
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_aucs.append(val_auc)
    test_aucs.append(test_auc)

    # always save epoch checkpoint
    torch.save(model.state_dict(), f"model_epoch_{epoch:02d}.pth")

    # best‑model logic
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        torch.save(model.state_dict(), best_model_path)
        epochs_since_improvement = 0
        print(f"✅  New best model (val AUC {val_auc:.4f})")
    else:
        epochs_since_improvement += 1
        print(f"⚠️   No improvement for {epochs_since_improvement} epoch(s)")

        if epochs_since_improvement >= 5:
            # revert & shrink LR
            model.load_state_dict(torch.load(best_model_path))
            for g in optimizer.param_groups:
                g["lr"] *= 0.5
            print(f"🔁  Reverted to best model, LR halved to {g['lr']:.6f}")
            epochs_since_improvement = 0

    # quick summary
    from sklearn.metrics import (
        accuracy_score,
        precision_score,
        recall_score,
        f1_score,
        classification_report
    )
    
    # Get predicted class labels
    val_preds = val_probs.numpy().argmax(axis=1)
    test_preds = test_probs.numpy().argmax(axis=1)
    val_true  = val_labels.numpy()
    test_true = test_labels.numpy()
    
    # --- Validation metrics ---
    val_acc      = accuracy_score(val_true, val_preds)
    val_prec     = precision_score(val_true, val_preds, average="macro", zero_division=0)
    val_rec      = recall_score(val_true, val_preds, average="macro", zero_division=0)
    val_f1       = f1_score(val_true, val_preds, average="macro", zero_division=0)
    
    # --- Test metrics ---
    test_acc     = accuracy_score(test_true, test_preds)
    test_prec    = precision_score(test_true, test_preds, average="macro", zero_division=0)
    test_rec     = recall_score(test_true, test_preds, average="macro", zero_division=0)
    test_f1      = f1_score(test_true, test_preds, average="macro", zero_division=0)
    
    # Print metrics summary
    print(f"Val Acc: {val_acc:.4f} | Prec: {val_prec:.4f} | Rec: {val_rec:.4f} | F1: {val_f1:.4f}")
    print(f"Test Acc: {test_acc:.4f} | Prec: {test_prec:.4f} | Rec: {test_rec:.4f} | F1: {test_f1:.4f}")
    print("Test Classification Report:")
    print(classification_report(test_true, test_preds, digits=4, zero_division=0))




Epoch 1/30


  0%|          | 0/1934 [00:00<?, ?it/s]

✅  New best model (val AUC 0.8928)
Val Acc: 0.9755 | Prec: 0.5588 | Rec: 0.5425 | F1: 0.5479
Test Acc: 0.9755 | Prec: 0.5582 | Rec: 0.5395 | F1: 0.5457
Test Classification Report:
              precision    recall  f1-score   support

           0     0.9832    0.9869    0.9851    340611
           1     0.8581    0.9756    0.9131     23694
           2     0.0000    0.0000    0.0000       218
           3     0.0000    0.0000    0.0000       128
           4     0.0000    0.0000    0.0000         3
           5     0.8422    0.9804    0.9061      1176
           6     0.9559    0.6390    0.7660       881
           7     0.9918    0.9821    0.9869     19244
           8     0.0000    0.0000    0.0000       278
           9     0.0000    0.0000    0.0000         4
          10     0.9551    0.8253    0.8855       876
          11     0.8444    0.8618    0.8530       825
          12     0.9941    0.8928    0.9407     34607
          13     0.9480    0.9486    0.9483      1635
         

  0%|          | 0/1934 [00:00<?, ?it/s]

✅  New best model (val AUC 0.8965)
Val Acc: 0.9783 | Prec: 0.5866 | Rec: 0.5654 | F1: 0.5709
Test Acc: 0.9785 | Prec: 0.5763 | Rec: 0.5638 | F1: 0.5686
Test Classification Report:
              precision    recall  f1-score   support

           0     0.9824    0.9913    0.9868    340611
           1     0.8962    0.9569    0.9255     23694
           2     0.0000    0.0000    0.0000       218
           3     0.0000    0.0000    0.0000       128
           4     0.0000    0.0000    0.0000         3
           5     0.9957    0.9864    0.9910      1176
           6     0.9438    0.8956    0.9190       881
           7     0.9966    0.9791    0.9877     19244
           8     0.0435    0.0036    0.0066       278
           9     0.0000    0.0000    0.0000         4
          10     0.9573    0.9224    0.9395       876
          11     0.8563    0.8812    0.8686       825
          12     0.9953    0.8913    0.9404     34607
          13     0.9779    0.9486    0.9631      1635
         

...

⚠️   No improvement for 5 epoch(s)
🔁  Reverted to best model, LR halved to 0.000250


/tmp/ipykernel_31/1857892331.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


Val Acc: 0.9947 | Prec: 0.7774 | Rec: 0.6896 | F1: 0.7012
Test Acc: 0.9945 | Prec: 0.7807 | Rec: 0.6590 | F1: 0.6819
Test Classification Report:
              precision    recall  f1-score   support

           0     0.9979    0.9954    0.9967    340611
           1     0.9918    0.9967    0.9942     23694
           2     1.0000    0.1193    0.2131       218
           3     0.0000    0.0000    0.0000       128
           4     0.0000    0.0000    0.0000         3
           5     0.9974    0.9923    0.9949      1176
           6     0.9609    0.9762    0.9685       881
           7     0.9997    0.9990    0.9994     19244
           8     0.9515    0.3525    0.5144       278
           9     0.0000    0.0000    0.0000         4
          10     0.9720    0.9897    0.9808       876
          11     0.8913    0.9842    0.9355       825
          12     0.9659    0.9983    0.9818     34607
          13     0.9823    0.9817    0.9820      1635
          14     1.0000    0.5000    0.6667 

  0%|          | 0/1934 [00:00<?, ?it/s]

✅  New best model (val AUC 0.9989)
Val Acc: 0.9954 | Prec: 0.7708 | Rec: 0.6902 | F1: 0.7021
Test Acc: 0.9953 | Prec: 0.8404 | Rec: 0.6767 | F1: 0.7095
Test Classification Report:
              precision    recall  f1-score   support

           0     0.9972    0.9971    0.9971    340611
           1     0.9923    0.9959    0.9941     23694
           2     0.9286    0.1193    0.2114       218
           3     0.0000    0.0000    0.0000       128
           4     0.0000    0.0000    0.0000         3
           5     0.9949    0.9898    0.9923      1176
           6     0.9623    0.9841    0.9731       881
           7     0.9997    0.9990    0.9993     19244
           8     0.9035    0.3705    0.5255       278
           9     1.0000    0.2500    0.4000         4
          10     0.9718    0.9852    0.9785       876
          11     0.8888    0.9879    0.9357       825
          12     0.9812    0.9914    0.9863     34607
          13     0.9852    0.9798    0.9825      1635
         